In [1]:
import pandas as pd
import numpy as np
import os
folder = os.getcwd()

In [2]:
triplets = pd.read_table(folder+'\\data\\10000.txt', header=None, names=['user_id', 'song_id', 'listen_count'])
metadata = pd.read_csv(folder+'\\data\\metadata.csv')

In [3]:
len(triplets)

2000000

In [4]:
# shuffle the data and only use 1% of it due to time and space constraints
triplets = triplets.sample(frac=0.01, random_state=2)

In [5]:
# 100000 rows out of 2000000 will be used
len(triplets)

20000

In [6]:
# verifying that considerable number of users have listened to atleast 5 songs, hence useful in item-item filtering
triplets.groupby(['user_id']).filter(lambda x: len(x) > 5)

,user_id,song_id,listen_count
1684075,a33c98d68030392526e4bef6d8126febbaf9be09,SOTCDXR12A679D7E5E,1
1594010,a15075a926c1998d91940f118342ba8356efc7d4,SOEBOWM12AB017F279,1
1756086,78f41a23ef3176c1d0a2fe59da2d0d511c689dd3,SOSJFDH12A8C143096,1
1428380,f82c096234fbadc0c7458b96fbacf000ba2d62b8,SOXVVSM12A8C142224,1
831928,5dffd87b491c26afe6b45ac0b9e60d4cf12e572e,SOCUBFV12AB01810BB,2
...,...,...,...
1594043,a15075a926c1998d91940f118342ba8356efc7d4,SOFLMUK12A8C132CDC,1
1826940,2b0bf093e23c6d98b94f0ceeff45230a7769414e,SOSPXWA12AB0181875,4
1297646,790cd3209a60dbe386545179e5ba8f61357fffe9,SORMGEO12A8C14406D,2
1297670,790cd3209a60dbe386545179e5ba8f61357fffe9,SOUFTBI12AB0183F65,2


In [7]:
len(triplets[triplets.user_id == '9fb0717a34c90c91ce09ab460969a8a428d3ac87'])

5

In [8]:
songs = pd.merge(triplets, metadata.drop_duplicates(['song_id']), on='song_id', how='left')

In [9]:
len(songs)

20000

In [10]:
len(songs)

20000

In [11]:
def full_name(title, artist):
    return (title + ' - ' + artist)
        
songs['name'] = songs.apply(lambda x: full_name(x['title'], x['artist_name']), axis=1)

In [12]:
songs['name']

0                 Shove It [feat. SpankRock] - Santogold
1              Loving You [Album Version] - Paolo Nutini
2                Far Behind  (Album Version) - Candlebox
3                     The Dirty Glass - Dropkick Murphys
4                           Tone Bank Jungle - Holy Fuck
                              ...                       
19995                      Throw Me A Rope - KT Tunstall
19996               Naturally - Selena Gomez & The Scene
19997                     Mr. Brownstone - Guns N' Roses
19998                             Toxic - Britney Spears
19999    Get Ready To Bounce Recall 08 - Brooklyn Bounce
Name: name, Length: 20000, dtype: object

In [13]:
songs_grouped = songs.groupby(['name']).agg({'listen_count': 'count'}).reset_index()
total_listens = songs_grouped['listen_count'].sum()
songs_grouped['percentage']  = songs_grouped['listen_count'].div(total_listens)*100
songs_grouped = songs_grouped.sort_values(by='percentage', ascending=False)

In [14]:
songs_grouped

,name,listen_count,percentage
5116,Sehr kosmisch - Harmonia,80,0.400
4859,Revelry - Kings Of Leon,76,0.380
6574,Undo - Björk,75,0.375
7155,You're The One - Dwight Yoakam,68,0.340
1473,Dog Days Are Over (Radio Edit) - Florence + Th...,61,0.305
...,...,...,...
3421,Les Garçons Dans Les Vestiaires - Clarika,1,0.005
3423,Leslie Anne Levine - The Decemberists,1,0.005
3428,Let It Go - Keyshia Cole / Missy Elliott / Lil...,1,0.005
3429,Let It Loose (Album Version) - The Rolling Stones,1,0.005


In [30]:
songs_grouped.to_csv(folder+'\\songs_grouped.csv')

In [30]:
songs.to_csv(folder+'\\songs_merged_20000.csv')